In [1]:
import numpy as np
import scipy.stats as si

In [2]:
St = 20 # Current stock price
K = 20  # Strike price
T = 1   # Time to maturity in years
r = 0.05 # Risk-free interest rate
sigma = 0.2 # Volatility of the underlying stock
t = 0   # Current time

In [3]:
def d_1(St, K, T, t, r, sigma):
    return (np.log(St / K) + (r + 0.5 * sigma ** 2) * (T - t)) / (sigma * np.sqrt(T - t))

def d_2(St, K, T, t, r, sigma):
    return (np.log(St / K) + (r - 0.5 * sigma ** 2) * (T - t)) / (sigma * np.sqrt(T - t))

d2 = d_2(St, K, T, t, r, sigma)
d1 = d_1(St, K, T, t, r, sigma)

d1, d2

(0.35000000000000003, 0.15)

In [4]:
def N(dn):
    return si.norm.cdf(dn, 0, 1)

N_d1 = N(d1)
N_d2 = N(d2)

N_d1, N_d2

(0.6368306511756191, 0.5596176923702425)

In [5]:
def N_d1_f(St, K, T, t, r, sigma, positive):
    d1 = d_1(St, K, T, t, r, sigma)
    if positive:
        Ndn = N(d1)
    else:
        Ndn = N(-d1)
    return Ndn

def N_d2_f(St, K, T, t, r, sigma, positive):
    d2 = d_2(St, K, T, t, r, sigma)
    if positive:
        Ndn = N(d2)
    else:
        Ndn = N(-d2)
    return Ndn

N_d1_total = N_d1_f(St, K, T, t, r, sigma, 1)
N_d2_total = N_d2_f(St, K, T, t, r, sigma, 1)

N_d1_total, N_d2_total

(0.6368306511756191, 0.5596176923702425)

# Binary CN

In [6]:
def BinCNPut(St, K, T, t, r, sigma):
    return K * np.exp(-r * (T - t)) * N_d2_f(St, K, T, t, r, sigma, 0)

def BinCNCall(St, K, T, t, r, sigma):
    return K * np.exp(-r * (T - t)) * N_d2_f(St, K, T, t, r, sigma, 1)

BinPutCN = BinCNPut(St, K, T, t, r, sigma)
BinCallCN = BinCNCall(St, K, T, t, r, sigma)

BinPutCN, BinCallCN

(8.378092180939014, 10.646496309075268)

# Binary AN

In [7]:
def BinANPut(St, K, T, t, r, sigma):
    return St * N_d1_f(St, K, T, t, r, sigma, 0)

def BinANCall(St, K, T, t, r, sigma):
    return St * N_d1_f(St, K, T, t, r, sigma, 1)


BinPutAN = BinANPut(St, K, T, t, r, sigma)
BinCallAN = BinANCall(St, K, T, t, r, sigma)

BinPutAN, BinCallAN

(7.263386976487618, 12.736613023512382)

# Financial options

In [8]:
def BinPut(St, K, T, t, r, sigma):
    return BinCNPut(St, K, T, t, r, sigma) - BinANPut(St, K, T, t, r, sigma)

def BinCall(St, K, T, t, r, sigma):
    return BinANCall(St, K, T, t, r, sigma) - BinCNCall(St, K, T, t, r, sigma)

put = BinPut(St, K, T, t, r, sigma)
call = BinCall(St, K, T, t, r, sigma)

put, call

(1.1147052044513952, 2.090116714437114)

# All options

In [11]:
def all_options(St, K, T, t, r, sigma):
    put_cn = BinCNPut(St, K, T, t, r, sigma)
    call_cn = BinCNCall(St, K, T, t, r, sigma)
    put_an = BinANPut(St, K, T, t, r, sigma)
    call_an = BinANCall(St, K, T, t, r, sigma)
    financial_put = put_cn - put_an
    financial_call = call_an - call_cn
    
    print(f"Binary Cash-or-Nothing Put Option Fair Price:   ${put_cn:.4f}")
    print(f"Binary Cash-or-Nothing Call Option Fair Price:  ${call_cn:.4f}")
    print(f"Binary Asset-or-Nothing Put Option Fair Price:  ${put_an:.4f}")
    print(f"Binary Asset-or-Nothing Call Option Fair Price: ${call_an:.4f}")
    print(f"Financial Put Option Fair Price:                ${financial_put:.4f}")
    print(f"Financial Call Option Fair Price:               ${financial_call:.4f}")

all_options(St, K, T, t, r, sigma)

Binary Cash-or-Nothing Put Option Fair Price:   $8.3781
Binary Cash-or-Nothing Call Option Fair Price:  $10.6465
Binary Asset-or-Nothing Put Option Fair Price:  $7.2634
Binary Asset-or-Nothing Call Option Fair Price: $12.7366
Financial Put Option Fair Price:                $1.1147
Financial Call Option Fair Price:               $2.0901
